<a href="https://colab.research.google.com/github/GoodLight999/webui_colab/blob/main/webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

・このノートは、camenduru氏の配布しているipynbを参考にして作られています。</br>
https://github.com/camenduru/stable-diffusion-webui-colab</br>
</br>
・権利者に無断で、画像生成AIにイラストを学習させる行為は、日本の著作権法30条の4第2号に則って完全に**合法**です。また、拡散モデルによる画像生成は、ニューラルネットを用いたノイズ除去が元になっており、**イラストのコラージュからは程遠い手法**です。しかしそれはそれとして**不快に思うイラストレーター**は居ます。</br>
また、そういった一部の人の気持ちを気にしなくとも、現在のアニメ調イラスト生成モデルには、NovelAIから**不正流出したデータを元に作られている限りなく黒に近いモデルが氾濫しているという非常に大きな問題**があります。</br>
加えてanbooruという英語による丁寧なタグ付けがなされた**イラスト転載サイトを学習元に使っているモデルも多い**です。法的にはクリーンなWaifu Diffusion もこの問題からは逃れられません。</br>
danbooru自体は、転載元をしっかり掲載したうえで画像を詳細にタグ付けしている非営利のサイトで、米国法で無断転載が合法となる**フェアユース**規定(Google画像検索も無断転載を検索結果に表示しているがそれが合法な理由)に則ろうとしている(比較的)誠実な転載サイトではあります。でも、無断であることに不快感を覚えるイラストレーターが居るのは事実。</br>
こういったことを念頭に置いて、画像を生成しましょう。</br>
</br>
・たぶん、</br>
**編集　→ノートブックの設定　→ハードウェア アクセラレータ　→GPU**</br>
と設定したあと</br>
**ランタイム　→全てのセルを実行**</br>
を押すだけで動きます！</br>
起動にはけっこう時間がかかります。8分近くかかるかも……</br>
PC推奨です。処理はGoogleのサーバーを借りて行うので、一応スマホからでも動かせます。ちょっと使いにくいけどね。

In [ ]:
#@title 起動準備
# aria2(超高速ダウンロードを並列化で実現する仕組み)を導入
!apt-get -y install -qq aria2

# xformers(画像生成高速化の仕組み)を準備
import os
from google.colab.output import eval_js
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!pip install -q --pre xformers
!pip install -q --pre triton

In [ ]:
#@title webuiの準備
# webuiを連れてくる
!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui

!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py

In [ ]:
#@title webui向けの拡張機能・追加ファイルの導入
# image briwser
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
# civitai browser
!git clone https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
# additional networks(for LoRA)
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
# tag complete
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete /content/stable-diffusion-webui/extensions/a1111-sd-webui-tagcomplete
# aspect ratio selector
!git clone https://github.com/alemelis/sd-webui-ar /content/stable-diffusion-webui/extensions/sd-webui-ar
# ControlNet
!git clone https://github.com/Mikubill/sd-webui-controlnet.git /content/stable-diffusion-webui/extensions/sd-webui-controlnet
# Openpose editor (for ControlNet)
!git clone https://github.com/fkunn1326/openpose-editor.git /content/stable-diffusion-webui/extensions/openpose-editor
# 日本語化
# !git clone  https://github.com/Katsuyuki-Karasawa/stable-diffusion-webui-localization-ja_JP /content/stable-diffusion-webui/extensions/stable-diffusion-webui-localization-ja_JP

# EasyNegative
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors
# EasyNegative
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt
# bad_prompt_version2.pt
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/datasets/Nerfgun3/bad_prompt/resolve/main/bad_prompt_version2.pt
# wdbadprompt.pt
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/waifu-diffusion/wd-1-5-beta/resolve/main/embeddings/wdbadprompt.pt
# wdgoodprompt.bin
!wget -P /content/stable-diffusion-webui/embeddings https://huggingface.co/waifu-diffusion/wd-1-5-beta/resolve/main/embeddings/wdgoodprompt.bin


# 生成完了時に通知音を鳴らすようにする
!wget https://otologic.jp/sounds/se/pre/Accent03-1.mp3 -O /content/stable-diffusion-webui/notification.mp3

# ディレクトリを移動
%cd /content/stable-diffusion-webui

!git reset --hard

In [ ]:
#@title  modelとVAEのダウンロード
# Stable Diffusion
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1_768-ema-pruned.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1/raw/main/v2-inference-v.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1_768-ema-pruned.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/joachimsallstrom/Double-Exposure-Embedding/resolve/main/dblx.pt -d /content/stable-diffusion-webui/embeddings -o dblx.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Conflictx/AnimeScreencap/resolve/main/AnimeScreenCap.pt -d /content/stable-diffusion-webui/embeddings -o AnimeScreenCap.pt

# AbyssOrangeMix2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AbyssOrangeMix2_hard.safetensors
# AbyssOrangeMix3
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A1_Flat,animelike.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1B_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A1B_illustration_between_A1andA3.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A2_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A2_Artistic,Oil_paintings_like.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A3_Midpoint_of_Artistic_and_Kawaii.safetensors
# AbyssOrangeMix用VAE
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -d /content/stable-diffusion-webui/models/VAE -o OrangeMix.vae.pt

# Counterfeit-V2.5
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o Counterfeit-V2.5.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt -d /content/stable-diffusion-webui/models/VAE -o Counterfeit-V2.5.vae.pt

# Grapefruit
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/9000 -d /content/stable-diffusion-webui/models/Stable-diffusion -o grapefruitHentaiModel_VAE_not_required.safetensors

# Pastel-Mix
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o pastelmix-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/pastel-mix/resolve/main/pastel-waifu-diffusion.vae.pt -d /content/stable-diffusion-webui/models/VAE -o pastelmix-fp16.vae.pt

# SukiyakiMix
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Vsukiyaki/SukiyakiMix-v1.0/resolve/main/SukiyakiMix-v1.0-fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o SukiyakiMix-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -d /content/stable-diffusion-webui/models/VAE -o SukiyakiMix-fp16.vae.pt

# 7th_Layer
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/syaimu/7th_Layer/resolve/main/7th_anime_v3/7th_anime_v3_B.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o 7th_anime_v3_B.safetensors

# Waifu Diffusion 
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/waifu-diffusion/wd-1-5-beta2/resolve/main/checkpoints/wd-1-5-beta2-fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o wd15-beta1-fp16.safetensors
# !wget https://huggingface.co/waifu-diffusion/wd-1-5-beta2/resolve/main/checkpoints/wd-1-5-beta2-fp16.yaml -O /content/stable-diffusion-webui/models/Stable-diffusionwd15-beta1-fp16.yaml.mp3
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o kl-f8-anime2.vae.pt

In [ ]:
#@title LoRAやControlNetもダウンロード

# Honkai Impact 3-真理之律者/Herrscher of Truth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/10757 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o honkaiImpact3Herrscher.safetensors
# LightningVFX - Create more consistent Lightning
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/12144 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o LightningVFX.safetensors
# Bondage Suspension
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/10347 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Bondage_Suspension.safetensors
# Milking machine
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/10069 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Milking_machine.safetensors
# Manhattan Cafe (umamusume)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/8935 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Manhattan_Cafe.safetensors
# Vibrator
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/12843 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Vibrator.safetensors
# Esthetic Futanari + Trap
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/OneRing/TP3.7.safetensors/resolve/main/TP3.7.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Futanari_Trap.safetensors
# Silence Suzuka (umamusume)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/13513 -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora -o Silence_Suzuka.safetensors

# ControlNetが使えるように、ファイルをダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth

In [ ]:
#@title  起動！</br>(辛抱強く5分位待つと、https://?????.gradio.liveみたいなリンクが表示されるので、それをクリックして下さい。)</br>(閉じるときは、ランタイム→ランタイムを接続解除して削除。)
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py

# 設定を予め変更する
import json
config = {
    "CLIP_stop_at_last_layers": 2,
    "quicksettings": "sd_model_checkpoint,sd_vae,CLIP_stop_at_last_layers"
}
with open("config.json", "w") as f:
    json.dump(config, f)

# 起動！！！ --no-half-vae
!python launch.py --share --xformers --no-half-vae --enable-insecure-extension-access --gradio-queue

# **日本語化について**
webuiは頻繁に機能が追加・更新されるので、それに追いつけてない不完全な日本語化パッチを当てるより英語のままのほうがググりやすくて便利な気がします。</br>
よって日本語化をしていません。ご容赦ください。</br>
「webui向けの拡張機能・追加ファイルの導入」に、日本語化パッチのインストール準備自体はしてあるので、コメントアウトのための#を消してインストールすれば、簡単に日本語化出来ます。

---

# **VAEとは**</br>
主に塗りに影響を与える、調整ファイル。</br>
これが不適切だと変に淡い塗りの画像が出力される。</br>
一応、AbyssOrangeMix以外は、設定をしなくても上手く動くようにしてあるはず。

---

# **導入モデルの簡易紹介**</br>
モデルの配布リンク先を見れば、プロンプトや使用Samplerが載ってる作例が見れるので、見てみてね。</br></br>
**Stable Diffusion**</br>
最も有名な画像生成AIの1つにして、様々な画像生成AIの大本になっている、Stable Diffusionの最新版モデル。</br>
リアル画像に強く、アニメ調イラストは弱い。。</br>
プロンプトの書き方が、以下に記されているアニメ調イラストの生成に特化したモデルたちと違うので注意が必要。</br>
解説サイト↓</br>
https://webbigdata.jp/post-14827/</br>
</br>
**AbyssOrangeMix**：流行ってるモデル。AOM3系が最新。</br>
ややAI臭いが美麗。</br>
R-18も可能。</br>
(このモデルだけ、VAEを自分で選択してもらう必要があるから、注意!)</br>
推奨Negative：(worst quality, low quality:1.4), (realistic, lip, nose, tooth, rouge, lipstick, eyeshadow:1.0), (dusty sunbeams:1.0),, (abs, muscular, rib:1.0), (depth of field, bokeh, blurry:1.4), (greyscale, monochrome:1.0), text, title, logo, signature</br>
推奨Sampler：DPM++ SDE Karras</br>
https://huggingface.co/WarriorMama777/OrangeMixs</br>
</br>
**Counterfeit**-V2.5：崩れづらい分使い手の個性を出しづらい。</br>
でも、キャラクターの顔が非常に崩れにくく、衣装や小物から背景まで、全てを上手に出力してくれる。</br>
個人的には、イラスト風味の背景を描くのはこのモデルが一番上手だと思う。</br>
R-18は一応は可能。</br>
推奨Negative：EasyNegative, (worst quality, low quality:1.4)</br>
推奨Sampler: DPM++ 2M Karras</br>
推奨CFG Scale：10 ←デフォの7より高めるとパキっと濃くなる</br>
https://huggingface.co/gsdf/Counterfeit-V2.5</br>
</br>
**Grapefruit**：エロ方向に先鋭化している。顔のAI臭さは強いが、スケベプロンプトの制御が効きやすい。</br>
リンク先にはR-18画像がバンバンあるので注意。</br>
(VAE不要！)</br>
推奨Sampler：DDIM, DPM++ SDE Karras</br>
https://civitai.com/models/2583/grapefruit-hentai-model</br>
</br>
**Pastel-Mix**：パステルカラーの独特なイラストを生成できる。AI臭さが非常に低い、個性派モデル。</br>
R-18は厳しいけど、このモデルにそんなことを期待するのもどうかと思う。</br>
推奨Sampler：DPM++ 2M Karras</br>
https://civitai.com/models/5414/pastel-mix-stylized-anime-model</br>
</br>
**SukiyakiMix**：Pstel-MixとAbyssOrangeMixを6:4でマージしただけのモデル。単純だけど、それだけに使い勝手が良い。</br>
推奨Negative：Negative:
(low quality, worst quality)1.4, (bad anatomy)+, (inaccurate limb)1.3,bad composition, inaccurate eyes, extra digit,fewer digits,(extra arms)1.2,logo,text</br>
https://huggingface.co/Vsukiyaki/SukiyakiMix-v1.0</br>
</br>
**Waifu Diffusion**</br>
法的にクリーンなモデル。現在最新は1.5beta。</br>
正直なところ、他のモデルよりも良い絵を出すのが難しいが、不正流出モデルの派生なんぞが覇権を取り続ける状況は好ましくないから、応援したい。</br>


---

# **プロンプト参考リンク**
**・使いたいモデルの配布サイト**</br>
作者が直々に看板として作った上手な作例が載ってるので、大いに参考にすべし。</br>
</br>
**・【NovelAI】 エロもいける！最強プロンプト（呪文）辞典**</br>
**・【ローカル版】エロ特化！最強プロンプト（呪文）新辞典★無料公開版★**</br>
この名前だけど、エロ目的じゃなくても非常に参考になる実践プロンプト集になっている。</br>
https://iosworld.blog.2nt.com/blog-entry-528.html</br>
https://iosworld.blog.2nt.com/blog-entry-558.html</br>
</br>
**・ちちぷい**</br>
AIイラスト版のPixiv。投稿作の多くにモデルやプロンプト、サンプラーが載っているので、参考になる。</br>
https://www.chichi-pui.com</br>
</br>
**・MajinAI**</br>
AIイラスト投稿サイト2。こっちはイラスト生成者向けのプロンプト紹介サイト。</br>
https://majinai.art/ja/index.php</br>
</br>
**・【随時更新】NovelAI/Waifu Diffusionを使う際に有用なサイト・リンク集(備忘録)**</br>
更新頻度・速度ともに高いサイト。</br>
https://economylife.net/links-for-novelai-waifu-diffusion/#gsc.tab=0


---



# **プロンプトを組む上でのプチアドバイス**
**・トークンの区切り**</br>
Stable Diffusionは、一度に75トークンまでしか扱えないので、長いプロンプトを指示すると分割されて実行されます。</br>
その際、プロンプトが断裂して（例えばsilver hairが、silverとhairに分けられて銀髪にならなくなって）しまったり、次の区切りの76トークン目が必要以上に強く影響したり、といった現象が発生します。</br>
これを防ぐために「BREAK」というワードを使って強制的にプロンプトを区切ることができます。</br>
　　例：1girl, silver hair, **BREAK**, hair ornament, ……</br>
BREAK直後のタグは強めに影響することを意識すると、より意図した絵を出しやすくなります。</br>
</br>
**・惜しい絵が出たとき、その細部を変える**</br>
Seedの隣りにあるExtraをチェックすると出てくる、VariationSeed・variation strengthの値をいじって再生成することで、その惜しい絵を微妙にズラすことが出来ます。
「variation seed」に惜しかった絵のSeed値を入れ、どれくらいそこからズラすかをvariation strengthで決めます。
0.1で様子見をして、強さを上下させて調整してみましょう。

**・惜しい絵が出たとき、その一部を変える**
出たイラストをSend to inpaint に送って、変更したい部分を黒塗りにして生成しましょう。</br>
もしくは、画像をダウンロードして変更したい部分を雑に色塗りして直して、それをinpaintにアップロードして変更箇所を黒塗りにして、生成しましょう。

---



# **Textual Inversion</br>**
わざと駄目なイラストを学習させたモデルをネガティブプロンプトから呼ぶことで、品質を向上させる技術があります。</br>
このコードではEasyNegativeというものを導入済みなので、ネガティブプロンプトの欄に**EasyNegative**と打つだけで、**強力な品質向上**が狙えます。</br>
Counterfeitというモデルの作者が作ったものなので、それと相性が良いです。デメリットとしては、ちょっと**リアル寄りの画風に引っ張られる節**もあります。</br>
bad_prompt_version2というものも入れてあるので、これも使えますが、EasyNegativeよりも絵柄への影響が強いです。

---



# **ControlNet**
話題のControlNetは導入済みなのですぐ使えます。</br>

---



# **LoRA**</br>
既存モデルに、なるべく低負荷で、画風やポーズ、特定キャラクターなどの追加要素を学ばせる仕組みとしてLoRAというものがあります。</br>
試験的に、</br>
・[崩壊3rdの真理の律者のデザインを追加するLoRAモデル](https://civitai.com/models/9101/honkai-impact-3-herrscher-of-truth)</br>
・[電撃を放てるようになるLightningVFX](https://civitai.com/models/10222/lightningvfx-create-more-consistent-lightning)</br>
などを組み込んでおきました。</br>
additional networkをEnableにして、モデルを選んで、weightを決めて、トリガーワードをプロンプトに組み込めば、使えます。</br>
コードをいじれば、他のLoRAモデルも導入できると思うので、興味があったら私に連絡するか、自分で頑張ってみてください。</br>
weightをどう設定するか迷ったら、とりあえず0.6(～0.8)くらいにしておくと良いよ。</br>
</br>
※LoRAが配布されているサイト</br>
https://civitai.com</br>
絞り込めばLoRAだけを見れます。</br>

| Loraモデル | トリガーになるプロンプト | 推奨weight |
| - | - | - |
| [真理の律者](https://civitai.com/models/9101/honkai-impact-3-herrscher-of-truth) | long_hair,drill_hair, breasts, symbol-shaped_pupils, ponytail,very_long_hair,multicolored_hair,</br>bare_shoulders,small_breasts, eyebrows_visible_through_hair, bangs, hair_between_eyes, |  |
| [LightningVFX](https://civitai.com/models/10222/lightningvfx-create-more-consistent-lightning) | \<lora:lightningvfxv1:0.2\> |  |
| [Manhattan Cafe (umamusume)](https://civitai.com/models/7607/manhattan-cafe-umamusume) | manhattan cafe \(umamusume\), white ahoge, black long hair</br>Racing competition clothes :black coat, black shirt, black skirt, yellow necktie | 0.5~0.65 |
| [Silence Suzuka (umamusume)](https://civitai.com/models/11415/silence-suzuka-umamusume)</br>NOT COMPATIBLE WITH AOM3A2, AOM3A3 model. | silence suzuka \(umamusume\)</br>black gloves, white skirt, pleated skirt, black pantyhose, layered sleeves, puffy short sleeves, short over long sleeves, black bow, mismatched footwear, asymmetrical footwear</br>(optional) smile. she is serious when running</br> | 0.7~1.0 |

<details>
  <summary>R-18のLoRA</summary>

  | Loraモデル | トリガーになるプロンプト | 推奨weight |
  | - | - | - |
  | [Bondage Suspension](https://civitai.com/models/8762/bondage-suspension) | suspension, ball gag, arms behind back |  |
  | [Milking machine](https://civitai.com/models/8500/milking-machine) | milking machine, lactation, nipples, leaning forward, |  |
  | [Vibrator](https://civitai.com/models/10820/vibrator) | vibrator on nipple, vibrator under clothes, vibrator in thighhighs, vibrator cord, egg vibrator, tape |  |
  | [Esthetic Futanari + Trap](https://civitai.com/models/7327/esthetic-futanari-trap-v37) | penis, futanari, otoko no ko, female focus, small penis, large penis (carefully, but work), medium penis|  |
  
</details>

# **便利な設定**</br>
自動で設定されるようにプログラムしておきました。</br>
</br>
~~**settings →Userinterface →Quicksetting list**（下の方にある） に</br>
sd_model_checkpoint,sd_vae,CLIP_stop_at_last_layers</br>
と書いて、</br>
**Apply settings →Reload UI**</br>
と押すと、UIの一番からVAEが変えられるようになります。</br>
エラーが起きますが、最後の起動コードを停止■させ、再度実行▶すれば大丈夫です。待ち時間も2回目は短いです。</br>
Clip Skipはまあ2にしておいて。俺もよくわからない。~~</br>

---